In [13]:
import pandas as pd 
from langchain_groq.chat_models import ChatGroq
import json
from sklearn.metrics import accuracy_score


In [14]:
with open("secrets.json") as f:
    secrets = json.load(f)   
Groq_Token = secrets["groq"]

In [15]:
groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}


In [27]:
acc1=pd.read_csv("./../../Combined/Train/LAYING/Subject_1.csv")
# System Prompts 

def zsl(acc1):
    query = f"""
    * You are a Human activity recognation model. 
    * Your task is to analyze the sentiment expressed in the given text and classify it as 'Laying', 'Sitting', 'Standing', "Walking', Walking_Downstairs or Walking_upstairs. 
    * Provide the Human activity through the acceleration in x, y and z direction.
    * Tell only the answer fast. No need for explanation.

    Acceleration: {acc1}
    """ 


    # To use Groq LLMs 
    model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
    answer = llm.invoke(query)

    return answer.content

print(zsl(acc1))

Walking_downstairs


In [33]:
def fsl(acc):
    acc1=pd.read_csv("./../../Combined/Train/LAYING/Subject_1.csv")
    acc2=pd.read_csv("./../../Combined/Train/SITTING/Subject_1.csv")
    acc3=pd.read_csv("./../../Combined/Train/STANDING/Subject_1.csv")
    acc4=pd.read_csv("./../../Combined/Train/WALKING/Subject_1.csv")
    acc5=pd.read_csv("./../../Combined/Train/WALKING_DOWNSTAIRS/Subject_1.csv")
    acc6=pd.read_csv("./../../Combined/Train/WALKING_UPSTAIRS/Subject_1.csv")
    # System Prompts 
    query = f"""
    * You are a Human activity recognation model. 
    * Your task is to analyze the sentiment expressed in the given text and classify it as 'Laying', 'Sitting', 'Standing', "Walking', Walking_Downstairs or Walking_upstairs. 
    * Provide the Human activity through the acceleration in x, y and z direction use the example to answer question at last.
    * Tell only the answer. No need for explanation.
    1. Acceleration: {acc1}
    Output = Laying
    2. Acceleration: {acc2}
    Output = Sitting
    3. Acceleration: {acc3}
    Output = Standing
    4. Acceleration: {acc4}
    Output = Walking
    5. Acceleration: {acc5}
    Output = Walking_downstream
    6. Acceleration: {acc6}
    Output = Walking_upstream

    Acceleration: {acc}
    Output = ?
""" 


    # To use Groq LLMs 
    model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
    answer = llm.invoke(query)

    return answer.content
acc= "../../Combined/Test/WALKING_DOWNSTAIRS/Subject_10.csv"
print(fsl(acc))

Walking_downstream


In [18]:
# import os
# folder = "../../Combined/Test/"
# folders = os.listdir(folder)
# # print(all_files)
# csv_files=[]
# for fol in folders:
#     print(fol)
#     all_files = os.listdir(fol)
#     csv_files += [file for file in all_files if file.endswith('.csv')]
#     for file_name in csv_files:
#         # Construct full file path
#         file_path = os.path.join(fol, file_name)
        
#         # Load the CSV file into a pandas DataFrame
#         data = pd.read_csv(file_path)

# # for file in folder:

In [19]:
# zsl_prediction

In [37]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score

# Assuming the code you provided is already defined:
# - zsl function
# - fsl function

# Folder containing the test data
folder = "../../Combined/Test/"
folders = os.listdir(folder)

# Lists to store actual labels and model predictions
actual_labels = []
zsl_predictions = []
fsl_predictions = []

for fol in folders:
    print(f"Processing folder: {fol}")
    all_files = os.listdir(os.path.join(folder, fol))
    csv_files = [file for file in all_files if file.endswith('.csv')]
    
    for file_name in csv_files:
        # Construct full file path
        file_path = os.path.join(folder, fol, file_name)
        
        # Load the CSV file into a pandas DataFrame
        data = pd.read_csv(file_path)
        
        # Assume the actual label is the folder name (e.g., 'SITTING', 'LAYING', etc.)
        actual_label = fol
        
        # Get predictions from zero-shot and few-shot models
        zsl_prediction = zsl(data)
        fsl_prediction = fsl(data)
        
        # Append the actual and predicted labels to the lists
        actual_labels.append(actual_label)
        zsl_predictions.append(zsl_prediction.strip().upper())
        fsl_predictions.append(fsl_prediction.strip().upper())

# Calculate the accuracy for zero-shot and few-shot predictions
zsl_accuracy = accuracy_score(actual_labels, zsl_predictions)
fsl_accuracy = accuracy_score(actual_labels, fsl_predictions)

# Output the accuracies
print(f"Zero-shot accuracy: {zsl_accuracy * 100:.2f}%")
print(f"Few-shot accuracy: {fsl_accuracy * 100:.2f}%")

Processing folder: LAYING
Processing folder: SITTING
Processing folder: STANDING
Processing folder: WALKING
Processing folder: WALKING_DOWNSTAIRS
Processing folder: WALKING_UPSTAIRS
Zero-shot accuracy: 20.37%
Few-shot accuracy: 44.44%


In [38]:
zsl_predictions
# fsl_predictions

['SITTING',
 'WALKING_DOWNSTAIRS',
 'WALKING_DOWNSTAIRS',
 'SITTING',
 'WALKING_UPSTAIRS',
 'WALKING',
 'WALKING',
 'SITTING',
 'WALKING_DOWNSTAIRS',
 'SITTING',
 'LAYING',
 'SITTING',
 'WALKING_DOWNSTAIRS',
 'SITTING',
 'STANDING',
 'SITTING',
 'LAYING',
 'STANDING',
 'SITTING',
 'SITTING',
 'SITTING',
 'SITTING',
 'LAYING',
 'SITTING',
 'SITTING',
 'STANDING',
 'LAYING',
 'WALKING_DOWNSTAIRS',
 'WALKING_DOWNSTAIRS',
 'SITTING',
 'WALKING_DOWNSTAIRS',
 'WALKING',
 'WALKING_DOWNSTAIRS',
 'WALKING_DOWNSTAIRS',
 'WALKING_UPSTAIRS',
 'WALKING_DOWNSTAIRS',
 'WALKING_DOWNSTAIRS',
 'SITTING',
 'WALKING_DOWNSTAIRS',
 'WALKING_DOWNSTAIRS',
 'WALKING',
 'SITTING',
 'SITTING',
 'SITTING',
 'WALKING_DOWNSTAIRS',
 'WALKING_DOWNSTAIRS',
 'WALKING_DOWNSTAIRS',
 'WALKING_DOWNSTAIRS',
 'WALKING_DOWNSTAIRS',
 'WALKING',
 'WALKING_UPSTAIRS',
 'WALKING_DOWNSTAIRS',
 'WALKING',
 'WALKING_DOWNSTAIRS']

### Which is better Zero shot learning or Few shot learning?

We have seen that few-shot learning have more Accuracy than the zero shot learning because in few shot lerning we give some example data which help the model to identify the human activity more corectly where as zero shot may be less trained and biased all the model may be biased. Few shot learning is better than the zero shot learning in given senerio.

### What are the limitation of Zero-shot and Few-shot learning?

Zero shot learning is heavily depend on the description provided. if relationship between accelerometer data and the activities are not well define then model give inaccurate output.